# Generativa Adversarial Networks

Las Generative Adversarial Networks (GANs) son un tipo de modelo de aprendizaje profundo generativo que consisten en dos redes neurales: una generadora y una discriminadora. La red generadora se encarga de crear nuevos datos (por ejemplo, imágenes), mientras que la red discriminadora se encarga de determinar si los datos creados por la red generadora son reales o falsos.

El entrenamiento de una GAN consiste en una competencia entre la red generadora y la red discriminadora: la red generadora intenta crear datos cada vez más convincentes, mientras que la red discriminadora intenta hacer una clasificación cada vez más precisa. A medida que se entrena la GAN, la red generadora se vuelve cada vez más hábil en crear datos realistas, y la red discriminadora se vuelve cada vez más hábil en detectar datos falsos.

En términos matemáticos, el problema se puede plantear como un juego entre dos jugadores: el generador y el discriminador. El generador trata de maximizar su función de pérdida, mientras que el discriminador trata de minimizarla. La función de pérdida es una medida de la capacidad del discriminador para diferenciar entre los datos reales y los falsos.

![](https://miro.medium.com/v2/resize:fit:828/format:webp/1*t78gwhhw-hn1CgXc1K89wA.png)

En términos de implementación, esto se logra usando un optimizador para actualizar los pesos de las redes en cada iteración, basándose en la gradiente de la función de pérdida con respecto a los pesos. Al final del entrenamiento, el generador ha aprendido a producir datos que se parecen a los datos reales, mientras que el discriminador ha aprendido a ser un buen detector de datos falsos. Para conseguirlo se usa la diferencia entre las distribuciones estadísticas de **Kullback-Lieber**.

A continuación veamos un ejemplos de código que implemeta una GAN en forma de una clase:

In [ ]:
#Primero cargamos las librerías que nos harán falta
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.layers import (Input, Dense, Reshape, Flatten, Dropout,
                                     BatchNormalization, Activation, ZeroPadding2D, LeakyReLU,
                                     UpSampling2D, Conv2D)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam,SGD

In [ ]:
class GAN:
    def __init__(self, image_shape, latent_dimension):
        self.image_shape = image_shape
        self.latent_dimension = latent_dimension
        
        # Definir el clasificador
        self.discriminator = self.build_discriminator()
  
        # Definir el generador
        self.generator = self.build_generator()
        
        input_generator= Input(shape=(self.latent_dimension,)) # También se llama Z o simplemente ruido
        generated_image = self.generator(input_generator)
        #Checking the validity of the generated image
        output_discriminator = self.discriminator(generated_image)
  
        #Defining the combined model of the Generator and the Discriminator
        self.gan = Model(input_generator, output_discriminator)
        self.gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002,0.5))
        
    def build_generator(self):
  
        model = Sequential()
  
        #Building the input layer
        model.add(Dense(128 * 8 * 8, activation="relu",
                        input_dim=latent_dimensions))
        model.add(Reshape((8, 8, 128)))
          
        model.add(UpSampling2D())
          
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.78))
        model.add(Activation("relu"))
          
        model.add(UpSampling2D())
          
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.78))
        model.add(Activation("relu"))
          
        model.add(Conv2D(3, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))
  
  
        #Generating the output image
        noise = Input(shape=(latent_dimensions,))
        image = model(noise)
        
        generator = Model(noise, image)
        return generator

    def build_discriminator(self):
  
        #Building the convolutional layers
        #to classify whether an image is real or fake
        model = Sequential()
  
        model.add(Conv2D(32, kernel_size=3, strides=2,
                         input_shape=image_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
          
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.82))
        model.add(LeakyReLU(alpha=0.25))
        model.add(Dropout(0.25))
          
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.82))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
          
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.25))
        model.add(Dropout(0.25))
          
        #La capa de salida
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))
  
        image = Input(shape=self.image_shape)
        validity = model(image)
        
        discriminator = Model(image, validity)
        discriminator.compile(loss='binary_crossentropy',
                              optimizer=Adam(0.0002,0.5),
                              metrics=['accuracy'])
        #Hacer al discriminador no entrenable para permitir al generador aprender del gradiente
        discriminator.trainable = False
        return discriminator

    
    def train(self,X, num_epochs=1500, batch_size=32, display_interval=150):
        losses=[]
        #Normalizing the input
        X = (X / 127.5) - 1.
          
  
        #Defining the Adversarial ground truths
        valid = np.ones((batch_size, 1))
  
        #Adding some noise 
        valid += 0.05 * np.random.random(valid.shape)
        fake = np.zeros((batch_size, 1))
        fake += 0.05 * np.random.random(fake.shape)
  
        for epoch in range(num_epochs):
              
            #Training the Discriminator
              
            #Sampling a random half of images
            index = np.random.randint(0, X.shape[0], batch_size)
            images = X[index]
  
            #Sampling noise and generating a batch of new images
            noise = np.random.normal(0, 1, (batch_size, latent_dimensions))
            generated_images = self.generator.predict(noise)
              
  
            #Training the discriminator to detect more accurately
            #whether a generated image is real or fake
            discm_loss_real = self.discriminator.train_on_batch(images, valid)
            discm_loss_fake = self.discriminator.train_on_batch(generated_images, fake)
            discm_loss = 0.5 * np.add(discm_loss_real, discm_loss_fake)
              
            #Training the Generator
  
            #Training the generator to generate images
            #which pass the authenticity test
            genr_loss = self.gan.train_on_batch(noise, valid)
              
            #Tracking the progress                
            if epoch % display_interval == 0:
                 self.display_images()
                    
    def display_images(self):
        r, c = 4,4
        noise = np.random.normal(0, 1, (r * c,self.latent_dimension))
        generated_images = self.generator.predict(noise)
  
        #Scaling the generated images
        generated_images = 0.5 * generated_images + 0.5
  
        fig, axs = plt.subplots(r, c)
        count = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(generated_images[count, :,:,])
                axs[i,j].axis('off')
                count += 1
        plt.show()               
    
        

Importante, fíjese que el discriminador se pone en `trainable = False` porque se está entrenando el modelo combinado, no el discriminador individual. El modelo combinado consiste en el generador y el discriminador combinados, donde el generador está tratando de engañar al discriminador para que prediga que las imágenes generadas son reales.

El objetivo es mejorar el generador para que genere imágenes más realistas, por lo que solo se entrena el generador en este momento. Sin embargo, el discriminador todavía es necesario para calcular la validación de las imágenes generadas y para proporcionar retroalimentación al generador. Por eso, se congela la capacidad de entrenamiento del discriminador y solo se entrena el generador.

In [ ]:
#Cargar los datos en está ocasión CIFAR10
# 10 posible clases de objetos
(X, y), (_, _) = keras.datasets.cifar10.load_data()
  
#Escoger una de las clases
X = X[y.flatten() == 8]

#Defining the Input shape
image_shape = (32, 32, 3)
          
latent_dimensions = 100

gan = GAN(image_shape, latent_dimensions)

gan.train(X)

In [ ]:
#Plotting some of the original images 
s=X[:40]
s = 0.5 * s + 0.5
f, ax = plt.subplots(5,8, figsize=(16,10))
for i, image in enumerate(s):
    ax[i//8, i%8].imshow(image)
    ax[i//8, i%8].axis('off')
          
plt.show()

In [ ]:
#Plotting some of the last batch of generated images
noise = np.random.normal(size=(40, latent_dimensions))
generated_images = gan.generator.predict(noise)
generated_images = 0.5 * generated_images + 0.5
f, ax = plt.subplots(5,8, figsize=(16,10))
for i, image in enumerate(generated_images):
    ax[i//8, i%8].imshow(image)
    ax[i//8, i%8].axis('off')
          
plt.show()